In [2]:
!pip install accelerate bitsandbytes datasets peft torch torchvision transformers trl==0.26.0 wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 42.4 MB/s eta 0:00:00


In [3]:
from stages import DPO, SFT
from utils import generate_model_responses
import wandb

In [4]:
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: os_j_es (os_j_es-byu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# 1) Supervised Fine Tuning

In [5]:
SAMPLE = 5000
EPOCHS = 10
BATCH = 4

In [6]:
sft = SFT()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
sft_model, metrics = sft.train(data_subsample_size=SAMPLE,
                               num_epochs=EPOCHS,
                               batch_size=BATCH,
                               project_name = 'DPO')

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

...

Truncating eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.069300
100,2.608800
150,2.489200
200,2.467300
250,2.474700
300,2.427000
350,2.422200
400,2.405500
450,2.353200
500,2.356500


Step,Training Loss
50,3.069300
100,2.608800
150,2.489200
200,2.467300
250,2.474700
300,2.427000
350,2.422200
400,2.405500
450,2.353200
500,2.356500


train/entropy,█▆▄▄▄▃▃▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▁▁▂▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
train/grad_norm,▄▄▃▂▆▅▄▇▅▃▁▃▃▄▅▃▆▃▃█▅▄▅▅▄▄▅▆▂▃▂▃▃▄▅▅▃▃▃▅
train/learning_rate,██▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁
train/loss,██▇▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▂▃▁▂▃▂▂▂▂▁▁▂▂▂▁▁▁▁▁▂▂▂
train/mean_token_accuracy,▁▃▂▂▂▃▂▅▃▄▅▅▅▆▄▅▅▅▆▆▆▆▇▆▆▇▅▆▇▇▇▇▅▇▆██▆▇▇
train/num_tokens,▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇█████
total_flos,1775720661811200.0
train/entropy,2.03496
train/epoch,10


In [8]:
sft.SFT_train_exec_time

'12.0 mins 38.590 secs'

# 2) Direct Preference Optimization

In [9]:
dpo = DPO(sft_model, sft.tokenizer)

In [11]:
β = 0.1
MAX_RESP_LEN = 32
SUBSAMPLE = 5000
EPOCHS = 15
LR = 1e-5

In [12]:
dpo_model, metrics = dpo.train(subsample_size=SUBSAMPLE,
                               num_epochs=EPOCHS,
                               batch_size=BATCH,
                               β = β,
                               max_resp_len = MAX_RESP_LEN,
                               lr = LR)

README.md: 0.00B [00:00, ?B/s]

harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

helpful-base/train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

helpful-online/train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

helpful-rejection-sampled/train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

Applying chat template to eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Step,Training Loss
50,0.689600
100,0.685700
150,0.718600
200,0.706100
250,0.732200
300,0.701200
350,0.702000
400,0.673000
450,0.685500
500,0.716800


Step,Training Loss
50,0.689600
100,0.685700
150,0.718600
200,0.706100
250,0.732200
300,0.701200
350,0.702000
400,0.673000
450,0.685500
500,0.716800


train/epoch,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇████
train/grad_norm,▆█▇▆▄▄▇▆▆▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,███▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/logits/chosen,▄▄▄▂▂▂▂▂▁▁▂▃▃▄▄▅▅▆▆▆▆▆▆▇▆▇▇▇▇▇▇█▇███████
train/logits/rejected,▄▄▃▂▂▁▁▃▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
train/logps/chosen,█▇▇▇▇▄▄▃▃▃▃▃▂▃▃▄▃▃▂▃▂▂▂▂▂▂▂▂▁▃▂▂▂▁▂▂▁▂▁▂
train/logps/rejected,████▇▅▅▄▅▄▃▄▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▁▁
train/loss,████▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/rewards/accuracies,▁▁▅▆▇▇██████████████████████████████████
+3,...


In [13]:
dpo.DPO_train_exec_time

'60.0 mins 27.702 secs'

In [15]:
_ = generate_model_responses(dpo_model, dpo.tokenizer, max_new_tokens=MAX_RESP_LEN, save_responses=True)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
